In [3]:
import logging.config
from typing import List
from dotenv import load_dotenv
from fastapi import FastAPI, Query, Request, Response, Depends, Header
from fastapi.responses import JSONResponse
from db.db_conf import get_db
from fastapi.middleware.cors import CORSMiddleware
from sqlalchemy.orm import Session
from models.user import User
import jwt
from logs.logs_config import LOGGING_CONFIG
import logging
from schemas.response_schemas.secured import SecuredResponse
import json
from fastapi import FastAPI
from sqlalchemy.orm import Session
import os
from dotenv import load_dotenv
from fastapi_utils.session import FastAPISessionMaker

import requests 
from models import Order, Promo, PromoStats, Stats, Stocks, KeyWordsStats, Product
import datetime
import pandas as pd
from time import sleep

load_dotenv()
from __future__ import print_function
from collections import defaultdict
from sqlalchemy.inspection import inspect

database_uri = os.environ.get('DATABASE_URL')

# session = Session()
sessionmaker = FastAPISessionMaker(database_uri)

def query_to_dict(rset):
    result = defaultdict(list)
    for obj in rset:
        instance = inspect(obj)
        for key, x in instance.attrs.items():
            result[key].append(x.value)
    return result

with sessionmaker.context_session() as db:
    try:
       stats = db.query(Stats).select_from(Stats).join(Product).join(User).all()
       df = pd.DataFrame(query_to_dict(stats))
    except Exception as e:
        print(e)
        print(e)
df



In [ ]:
df.to_excel('df_pavel.xlsx')

In [1]:
from catboost import CatBoostRegressor



In [2]:
df

NameError: name 'df' is not defined

### 1. Обрабатываем фичи

In [32]:
df.dtypes

id                             int64
date                  datetime64[ns]
nmId                           int64
product_id                     int64
vendorCode                    object
brandName                     object
openCardCount                  int64
addToCartCount                 int64
ordersCount                    int64
ordersSumRub                 float64
buyoutsCount                   int64
buyoutsSumRub                float64
cancelCount                    int64
cancelSumRub                 float64
addToCartPercent             float64
cartToOrderPercent           float64
buyoutsPercent               float64
avg_price_rub                float64
user_id                        int64
dtype: object

In [ ]:
# выделяем год, месяц, день недели и убираем колонку с полной датой
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df = df.drop(columns='date')
df.head()

In [ ]:
# one hot encoding категориальных фичей, чтобы модель могла их обработать
df = pd.get_dummies(df, columns=['vendorCode', 'brandName'])

### 2) Создаем тренировочный и тестовый датасеты

In [ ]:
# делим на матрицу признаков и таргет (число заказов)
X = df.drop(columns='ordersCount')
y = df['ordersCount']

In [ ]:
# делим на тренировочную и тестовую выборку
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

### 3) Обучаем модель градиентного бустинга CatBoost

In [ ]:

from catboost import CatBoostRegressor